In [1]:
from bio_files_processor import OpenFasta
from das_biotools import run_genscan, DNASequence
from custom_random_forest import RandomForestClassifierCustom

from sklearn.datasets import make_classification

In [2]:
%reload_ext autoreload
%autoreload 2

# 1. Custom Random Forest

In [3]:
X, y = make_classification(n_samples=100000)
n_job_1_random_forest = RandomForestClassifierCustom(
    max_depth=30, n_estimators=10, max_features=2, random_state=42
)
n_job_2_random_forest = RandomForestClassifierCustom(
    max_depth=30, n_estimators=10, max_features=2, random_state=42
)

In [4]:
# %time n_job_1_random_forest.fit(X,y,n_jobs=1)

%time n_job_2_random_forest.fit(X,y,n_jobs=2)

CPU times: user 609 ms, sys: 1.31 s, total: 1.92 s
Wall time: 27 s


In [10]:
for i, tree in enumerate(n_job_2_random_forest.trees[0]):
    print(i)
    print
    print(tree)
    break

0
DecisionTreeClassifier(max_depth=30, max_features=2, random_state=42)


In [22]:
import numpy as np
import functools
from itertools import repeat
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier

from concurrent.futures import ProcessPoolExecutor


def sum_four(a, b, c, d):
    return d


a, b, c = 1, 2, 3

all_d_values = [1, 2, 3, 10]

# partial_sum_four = functools.partial(sum_four, a, b, c)

with ProcessPoolExecutor(max_workers=2) as pool:
    res = list(pool.map(sum_four, repeat(a), repeat(b), repeat(c), all_d_values))

res

Process SpawnProcess-25:
Process SpawnProcess-26:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/concurrent/futures/process.py", line 246, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'sum_four' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/multiprocessing/process.py", line 31

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [26]:
repeat(a)

repeat(1)

In [28]:
%%time
import requests

n_requests = 10
n_proc = 10
repeat("https://google.com/")
with ProcessPoolExecutor(n_proc) as pool:
    results = pool.map(requests.get, repeat("https://google.com/"))
    print(type(results), list(results)[:5])

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x111a1fbf0>>
Traceback (most recent call last):
  File "/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [5]:
%time n_jobs_1_pred = n_job_1_random_forest.predict(X, n_jobs=1)

%time n_jobs_2_pred = n_job_2_random_forest.predict(X, n_jobs=2)

In [10]:
n_jobs_1_pred == n_jobs_2_pred

CPU times: user 798 µs, sys: 780 µs, total: 1.58 ms
Wall time: 1.44 ms
CPU times: user 726 µs, sys: 494 µs, total: 1.22 ms
Wall time: 976 µs


/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/m.trofimov/mambaforge/envs/ds/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/m.trofimov/Dropbox/Study/BI/pyth/das_biotools/custom_random_forest.py:67: RuntimeWarning: invalid value encountered in cast
  return np.mean(predictions, axis=0).astype(int)


-9223372036854775808